In [131]:
import numpy as np

def k(x, y, sig=1):
    r = np.linalg.norm(x - y)
    return np.exp(-0.5 * np.square(r) / np.square(sig))

def GaussianKernel(X, sigma=1):
    n = X.shape[0]
    kernel = np.empty((n,n))
    for i in range(n):
        for j in range(n):
            kernel[i, j] = k(X[i], X[j], sigma)
    return kernel / n

def get_approximation(A, r):
    U, S, Vh = np.linalg.svd(A, full_matrices=True)
    return U[:, :r] @ np.diag(S[:r]) @ Vh[:r, :]

In [133]:
def _truncated_psd_root(M, rank=None):
    lamb, U = np.linalg.eigh(M)

    idx = lamb.argsort()[::-1]

    if rank:
        lamb = lamb[idx][:rank]
        U = U[:,idx][:,:rank]
    else:
        lamb = lamb[idx]
        U = U[:,idx]

    lamb = np.diag(lamb)

    M = U @ np.sqrt(lamb)

    return M

def recover_nystrom_edg(D, m: int, r: int, sampling_rate: float = 0.4):
    """
    Input:
        D:  m+n by m+n distance matrix
        m:  Nystrom parameter (# rows/cols)
        r:  ground truth dimension of points that 
            created D
        sampling_rate: rate at which to sample elements of F in D

    Returns: 
        K: estimate for the kernel matrix.
    """
    n = D.shape[0] - m

    E = D[:m, :m]
    F = D[:m, m:]

    # We use the particular centering from Platt
    # which lets us recover A from only E.
    J = np.eye(m)-(1/m)*np.ones((m,m))
    
    # sanity check: A = P[:m,:] @ P[:m,:].T agrees with this
    A = -0.5*((J @ E) @ J)
    
    # The last row of F is considered known, so we 
    # only sample the first (m-1) rows... 
    F_mask = np.random.binomial(1, sampling_rate, (m-1, n))
    
    # ...then add on a zero row so the sizes match.
    F_mask = np.vstack((F_mask, np.zeros(n)))
    
    # Using our mask for F, 
    # make indices we'll need for the CVX constraints
    I,J = np.nonzero(F_mask)
    
    M = np.repeat(m-1, J.size)
    
    # make constant matrix C for row offsets
    E_sums = (1/m)*np.sum(E, axis=1)
    
    # subtract off the last sum
    E_sums -= E_sums[-1]
    
    # m x n
    C = np.tile(E_sums[:,np.newaxis], n)

    # -- Start CVX program

    # X is (r, m), P is (m+n, r), P = [X' ; Y']
    X = P[:m, :].T

    Y = cvx.Variable(shape=(r,n))

    Xty = X.T @ Y

    constraints = [Xty[I, J] - Xty[M, J] == -0.5*(F[I, J] - F[M, J]) + 0.5 * C[I, J]]

    cost = cvx.norm(X.T @ Y, "nuc")

    p = cvx.Problem(cvx.Minimize(cost), constraints)
 
    p.solve()

    if p.status in ["infeasible", "unbounded"]:
        raise Exception(p.status)

    # -- End CVX program

    B = X.T @ Y.value

    C = (B.T @ np.linalg.pinv(A)) @ B

    # K = [A B; B' C];
    K = np.block([[A, B], [B.T, C]])

    return K

def evaluate_recovery(K, P, show_plot=False):
    """
    Input:
        K: estimated kernel matrix, (m+n, m+n)
        P: ground truth points, (m+n, r)
        show_plot: plot recovery comparison? (r=2 only)

    Output:
        RMSE
    """
    r = P.shape[1]

    show_plot = (show_plot and r < 3)

    # Eigendecomposition to recover points
    P_cvx = _truncated_psd_root(K, rank=r)
    
    # Procrustes analysis to align points
    R, _ = orthogonal_procrustes(P_cvx, P)

    P_cvx = P_cvx @ R
    
    RMSE = np.sqrt(np.mean(np.sum((P_cvx - P)**2, axis=1)))

    if show_plot:
        import matplotlib.pyplot as plt

        plt.scatter(P_cvx[:,0], P_cvx[:, 1], 10, 'r', '.')
        plt.scatter(P[:,0], P[:, 1], 10, 'b', 'o')
       
        plt.show()

    return RMSE

m = 50
n = 200
r = 2
sampling_rate = 0.4
show_plot = False

P = np.random.random((m+n, r))

# This centering assumes the Platt choice of s.
# Sanity check: the reconstruction is indeed worse
# if you don't do this.
P = P - np.mean(P[:m,:], axis=0)
D = squareform(pdist(P))**2
K = recover_nystrom_edg(D, m, r, sampling_rate)
RMSE = evaluate_recovery(K, P, show_plot)
print(RMSE)

2.567526011781179e-09


In [211]:
m, n, r = 200, 50, 100
X = np.random.random((m, n))
K = GaussianKernel(X)
# print(np.linalg.matrix_rank(K))
# print(np.linalg.eig(K))

A = K[:r, :r]
B = K[:r, r:]
C = K[r:, r:]
# print(B.T @ A)
A_tilda = get_approximation(A, r)
C_tilda = B.T @ np.linalg.pinv(A_tilda) @ B
eps = np.linalg.norm(C - C_tilda, ord='fro')
# D = C - C_tilda
# print(C)
# print(max(D.min(), D.max(), key=abs))
# print(eps)
print(eps / np.linalg.norm(C, ord='fro'))

# TODO

0.9511909796545401
